*please execute the cell below before starting the tutorial by selecting it and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit
from aiida import load_profile

load_profile()
!verdi daemon start

# Input generator code

On Monday, you learned about simplified input file for the FLEUR code. The inpgen code generates an `inp.xml` input-file for FLEUR using the simplified input. In AiiDA, a `Code` object representing input generator has to have a similar behaviour: it has to generate a `FleurinpData` from a list of simplified parameters. The code class that implements this is called `FleurinputgenCalculation` and in this tutorial we are going to learn how to use it in AiiDA interface.

First things first - let us get familiar with inputs of the FleurinputgenCalculation.

## Inputs of the input calculation

Input generator input nodes has only 4 main input nodes that are listed in the table below:

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Inpgen code| yes |
|structure | StructureData | Structure data node| yes |
|parameters | Dict | FLAPW parameters| no |
|settings | Dict | special settings | no |

**code** is a `Code` representing inpgen inself in the database. 

Simplified input file is represented as a pair of input nodes: **structure** and **parameters**. **structure** is a `StructureData` node to be inserted into the simpified input file (`&lattice`). **parameters** is a nested dictionary containing nested name: value pairs and representing all other parameters of the simplified input file. For instance, if one wants to add a line

    &kpt
      div1=24   div2=20   div3=1 /

in the simplified input file, it is necessary to append

    'kpt': {
        'div1': 24,
        'div2' : 20,
        'div3' : 1
        }

to the **parameters** dictionary.

In some cases the simplified input file does not have names for nested values. In this case you can refer to the list of all supported keys:

    'input': ['film', 'cartesian', 'cal_symm', 'checkinp', 'symor', 'oldfleur']
    
    'atom': ['id', 'z', 'rmt', 'dx', 'jri', 'lmax', 'lnonsph', 'ncst', 'econfig', 'bmu', 'lo', 'element', 'name']
             
    'comp': ['jspins', 'frcor', 'ctail', 'kcrel', 'gmax', 'gmaxxc', 'kmax']
    
    'exco': ['xctyp', 'relxc'],
    
    'film': ['dvac', 'dtild'],
    
    'soc': ['theta', 'phi'],
    
    'qss': ['x', 'y', 'z'],
    
    'kpt': ['nkpt', 'kpts', 'div1', 'div2', 'div3','tkb', 'tria'],
    
    'title': {}
                        
For example, you can find that SOC ``theta`` and ``phi`` values can be set appending

    'soc': {
        'theta': 0.7,
        'phi' : 0.26
        }

to the **parameters** dictionary.

**settings** is used to modify a list of files to be copied to/from/between calculation folders and to add command options to the executable. We will not cover this in the tutorial.


## Inpgen code submission

### Inputs preparation 

To sumbit an inpgen calculation, we need to import `FleurinputgenCalculation` class

In [ ]:
from aiida_fleur.calculation.fleurinputgen import FleurinputgenCalculation

and prepare all the inputs. Configured and ready-to-use inpgen code node is already stored in your database as pk=8. Now you can simple load it from the database:

In [ ]:
inpgen_code = load_node(2688)

We are going to use a structure that you created in tutorial 1. Please, find a pk of the Fe monolayer structure and load it via the `load_node()` command:

In [ ]:
structure_FeML = load_node(6204)

Finally, we need to set up a **parameters** dictionary. For now we are going to specify k-mesh and kmax only:

In [ ]:
parameters = Dict(dict={
    'comp': {
        'kmax': 3.85,
        },
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

In [ ]:
options = {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 1}, 'withmpi' : False}

### Assembling inputs in a single dictionary

The most convinent way to pass all the inputs into any calculation is to use its builder:

In [ ]:
inputs = FleurinputgenCalculation.get_builder()
inputs.parameters = parameters
inputs.code = inpgen_code
inputs.structure = structure_FeML
inputs.metadata.options = options

### Submission


To submit the input generator, run `submit` function passing all the input parameters:

In [ ]:
res_pk = submit(FleurinputgenCalculation, **inputs)
print('The PK of submitted job is'.format(res_pk)

`res_pk` contains a PK of the input generator calculation node stored in the database.

## Results analysis

After inpgen code is submitted, you can check the status of all running processes:

In [ ]:
!verdi process list -a -p 1

You can also check the status of inpgen calculation directly. Try running next cell and examine the output:

In [ ]:
!verdi process show 6231

Since input generator takes few second to finish, you most probably will not be able to catch it in any other state except 'Finished'. Now let us explore the output nodes. `Fleurinpgencalculation` generates three output nodes:

|name| type | comment|
|:--:|:----:|:----------:|
|fleurinpData|FleurinpData| represents `inp.xml`|
|remote_folder| FolderData| represents calculation folder |
|retrieved| FolderData | represents retrieved folder |

The most important is `fleurinpData` representing `inp.xml` file. To load it, run:

In [ ]:
inpgen_calc_node = load_node(res_pk)
output_fleurinp = inpgen_calc_node.outputs.fleurinpData

This `FleurinpData` can be used as an input for Fleur calculation which will be covered in next tutorial.

# Scripting tasks

Jupyter notebooks are convinient way to learn and run some tasks. However, I personally prefer using and running python scripts in a terminal for my work. As a final task of each tutorial, beggining from this one, you will be asked to construct a small python sript and run it in a terminal.

In this sub-section you find scripting tasks to be executed in a terminal.

First, you need to start a new terminal and type in:

     $workon aiida
     
To execute a python-script, run

     $(aiida) verdi run NAME_OF_THE_SCRIPT.py

#### 1.  Run an inpgen calculation for the Si

Use an existing inpgen_si.py file to write a script that submits `Fleurinpgencaluation` for a Si stucture given in the attached `Si.cif` file. Try to set up **parameters** dictionary to use parameters:

| name | value | comment |
|:----:|:-----:|:-------:|
| atom -> rmt  | 2.23  | muffin-tin radius |
| comp -> kmax | 3.84  | plane wave cut-off|
| kpt -> div1  | 2 | number of kpts along x|
| kpt -> div2  | 2 | number of kpts along y|
| kpt -> div3  | 2 | number of kpts along z|



Try finding out the PK of an output `FleurinpData` and remember it - you will need it in the next tutorial!